In [1]:
#Loading the MobileNet Model from Keras Library
from keras.applications import MobileNet

# MobileNet Model takes input images of 224x224 pixels
row, col= 224, 224 

# Re-loading the MobileNet model without the top  layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape =( row, col, 3))

#Freezing the layers. By default, they are trainable
for layer in MobileNet.layers:
    layer.trainable = False

# Printing the details of the models's layers 
print("*****************************")
print("List of all the Layers in the model : \n")
for (n,layer) in enumerate(MobileNet.layers):
    print(str(n) + " : "+ layer.__class__.__name__, layer.trainable)

#This function is to create the Top Layer for the model which will be placed above the bottom layers
def create_head(bottom_layers, num_classes):
    top_layer = bottom_layers.output
    top_layer = GlobalAveragePooling2D()(top_layer)
    top_layer = Dense(1024,activation='relu')(top_layer)
    top_layer = Dense(1024,activation='relu')(top_layer)
    top_layer = Dense(512,activation='relu')(top_layer)
    top_layer = Dense(num_classes,activation='softmax')(top_layer)
    return top_layer

#Placing the Top layer above the Bottom Layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 4
num_classes = 4
FC_Head = create_head(MobileNet, num_classes)
model = Model(inputs = MobileNet.input, outputs = FC_Head)
print("*****************************")
print("Model's Summary : \n")
print(model.summary())

### Loading our Dataset containing face Images
from keras.preprocessing.image import ImageDataGenerator

#**************************************
train_data_dir = 'Dataset/train/'
validation_data_dir = 'Dataset/validation/'
#**************************************

#AUGMENTATION 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Specify Batch Size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(row,col),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(row, col),
        batch_size=batch_size,
        class_mode='categorical')

### Training out Model
#Note we're using checkpointing and early stopping

from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
             
checkpoint = ModelCheckpoint("face_recognition_model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',optimizer = RMSprop(lr = 0.001), metrics = ['accuracy'])

# Enter the number of training and validation samples here
#**************************************
nb_train_samples = 62
nb_validation_samples = 3
#**************************************

# We train 10 EPOCHS because out dataset is small
epochs = 10
batch_size = 30

print("*****************************")
print("EPOCHS : ")
history = model.fit_generator(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = epochs, callbacks = callbacks, validation_data = validation_generator, validation_steps = nb_validation_samples // batch_size)

Using TensorFlow backend.
E:\SOFTWARES\AnacondaInstall\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\SOFTWARES\AnacondaInstall\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\SOFTWARES\AnacondaInstall\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\SOFTWARES\AnacondaInstall\lib\site-packages\tensorflow\python\fra

*****************************
List of all the Layers in the model : 

0 : InputLayer False
1 : ZeroPadding2D False
2 : Conv2D False
3 : BatchNormalization False
4 : ReLU False
5 : DepthwiseConv2D False
6 : BatchNormalization False
7 : ReLU False
8 : Conv2D False
9 : BatchNormalization False
10 : ReLU False
11 : ZeroPadding2D False
12 : DepthwiseConv2D False
13 : BatchNormalization False
14 : ReLU False
15 : Conv2D False
16 : BatchNormalization False
17 : ReLU False
18 : DepthwiseConv2D False
19 : BatchNormalization False
20 : ReLU False
21 : Conv2D False
22 : BatchNormalization False
23 : ReLU False
24 : ZeroPadding2D False
25 : DepthwiseConv2D False
26 : BatchNormalization False
27 : ReLU False
28 : Conv2D False
29 : BatchNormalization False
30 : ReLU False
31 : DepthwiseConv2D False
32 : BatchNormalization False
33 : ReLU False
34 : Conv2D False
35 : BatchNormalization False
36 : ReLU False
37 : ZeroPadding2D False
38 : DepthwiseConv2D False
39 : BatchNormalization False
40 : ReLU Fa

conv_dw_12 (DepthwiseConv2D) (None, 7, 7, 512)         4608      
_________________________________________________________________
conv_dw_12_bn (BatchNormaliz (None, 7, 7, 512)         2048      
_________________________________________________________________
conv_dw_12_relu (ReLU)       (None, 7, 7, 512)         0         
_________________________________________________________________
conv_pw_12 (Conv2D)          (None, 7, 7, 1024)        524288    
_________________________________________________________________
conv_pw_12_bn (BatchNormaliz (None, 7, 7, 1024)        4096      
_________________________________________________________________
conv_pw_12_relu (ReLU)       (None, 7, 7, 1024)        0         
_________________________________________________________________
conv_dw_13 (DepthwiseConv2D) (None, 7, 7, 1024)        9216      
_________________________________________________________________
conv_dw_13_bn (BatchNormaliz (None, 7, 7, 1024)        4096      
__________

In [ ]:
#Importing Libraries
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Loading our model
from keras.models import load_model
classifier = load_model('face_recognition_model.h5')

#VALIDATION
#**************************************
Default_Names = {"[0]": "EdSheeran", "[1]": "Shawn_Mendez","[2]": "ZyanMalik"}
Predicted_Names = {"n0": "Ed_Sheeran ",  "n1": "Shawn_Mendes","n2": "Zyan_Malik"}
#**************************************

#Configuring the Output : Image size and text/font
def Output_Config(name, pred, im):
    Name = Default_Names[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, Name, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

    
# This function will load random images from a random folder of our Validation/testing Dataset
def getRandomImage(path):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Predicted_Class : " + Predicted_Names[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    #Calling The Random image loading function
    input_im = getRandomImage("Dataset/validation/") 
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
     #Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    #Calling the output displaying function
    Output_Config("Model's_Prediction", res, input_original) 
    cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
from keras.models import load_model
classifier = load_model('face_recognition_model.h5')
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "EdSheeran", 
                      "[1]": "Shawn_Mendez",
                      "[2]": "ZyanMalik",
                     }

monkey_breeds_dict_n = {"n0": "EdSheeran ", 
                      "n1": "Shawn_Mendez",
                      "n2": "ZyanMalik",
                       }



def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Dataset/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Shawn_Mendez
Class - ZyanMalik
Class - Shawn_Mendez
Class - Shawn_Mendez
Class - EdSheeran 
Class - EdSheeran 
Class - Shawn_Mendez
Class - EdSheeran 
Class - EdSheeran 


KeyError: '.ipynb_checkpoints'